In [3]:
from camel.utils import print_text_animated
from camel.agents import ChatAgent
from camel.messages import BaseMessage

In [16]:

def refine_user_input(max_steps=10) -> str:
    refinement_msg = BaseMessage.make_assistant_message(
        "Story arch writer",
        "You are a creative writer focused on writing story arch for dnd one-shots. "
        "You create broad overviews of story archs. For you important is that each story has the following features."
        "A plot hook, something that the player want to help or solve the problem"
        "The environment, where the story is told."
        "Additionally these elements could improve the story."
        "Surprice Elements are also fun but optional, it is something the player are not expecting."
        "Interesting Characters, brings depth and live to the story"
        "A thread, something the players are eager to solve, whether it is a social conflict or existencial thread to a village you are open to it all."
    )
    refinement_agent = ChatAgent(refinement_msg)

    user_msg = input("Please specify your story.")

    initial_user_str = (
            "I will provide a description of a story arc."
            "The input should cover all information necessary to build the story. It should fullfill your important features."
            "If not then ask further questions about the story."
            "Never forget: Do not summarize the story until asked."
            "Never forget: If you have not further questions respond with '<NO_QUESTION>'"
            "Here is the user message:"
            + "{{" + user_msg + "}}")

    initial_user_msg = BaseMessage.make_user_message("User", initial_user_str)
    response = refinement_agent.step(initial_user_msg, )

    for i in range(max_steps):
        if i == max_steps - 1:
            break
        if "<NO_QUESTION>" in response.msg:
            break

        # print_text_animated(response.msg.content)
        print(response.msg.content)

        refined_user_input = input("Please answer the given questions.")
        refined_user_msg = BaseMessage.make_user_message(
            "User",
            "Here is an update one the story we build."
            + "{{" + refined_user_input + "}}")
        response = refinement_agent.step(refined_user_msg)

    result = refinement_agent.step(
        BaseMessage.make_user_message(
            "User",
            "Give a story overview with the information give by the user."
            "Structure the summary in the following blocks: player hook, story summary, important characters."
            "The story summary should give detailed information about necessary story elements."))

    # print_text_animated("Thank you for your input. Here is what i got: " + result.msg.content)
    print("Thank you for your input. Here is what i got: " + result.msg.content)
    return result.msg.content
story = refine_user_input()

To create a compelling story arc based on your description, I need to gather more details. Here are some questions to help flesh out the story:

1. **Plot Hook**: What motivates the Hase (hare) and Igel (hedgehog) to find the giant's fingernail? Is there a reward, a curse to break, or a personal reason for them?

2. **Environment**: What kind of bar do they meet in? Is it a magical tavern, a rustic inn, or something else? What is the surrounding area like?

3. **Interesting Characters**: Are there any other characters in the bar or in the story that could add depth? Perhaps other patrons, a bartender with secrets, or the giant themselves?

4. **Thread**: What is the main conflict or thread that the players will be eager to solve? Is it related to the giant, the fingernail, or something personal to the Hase and Igel?

5. **Surprise Elements**: Are there any unexpected twists or surprises you would like to include in the story? For example, a hidden agenda, a betrayal, or a magical pheno

In [17]:
structure_msg = BaseMessage.make_assistant_message(
    "Scene creator",
    "You are outlining the story in steps. Each step/encounter drives the story forward or reveal more information."
    "You have three broad encounter options: combat, NPC interaction, and exploration (traps, puzzles, navigation). The goal is to make sure you don't have too much of the same stuff back-to-back."
)
structure_agent = ChatAgent(structure_msg)

firstLastResponse = structure_agent.step(
    BaseMessage.make_user_message(
        "User", 
        "Given the story, define the first and last scene/encounter."
        "Each scene should be described in one sentence and should be labeled with an encounter option."
        "For example: "
        "- Follow the thief through the busteling city alleys (navigation)"
        "- Sorcerous orbs speaking in riddles (puzzle)"
        "- Secretive librarian (NPC interaction)"
        "- Final fight with thief and his friends (combat)"
        "Here is the story: "
        "{{" + story + "}}"
        "Define the first and last scene/encounter in the format of:"
        "Story Start: <first scene>"
        "Story End: <last scene>"))
print(firstLastResponse.msg.content)

rows = firstLastResponse.msg.content.split("\n")


Story Start: - Gather information about the Igel's curse and the giant Riespunzel from the lively patrons of "The Woodland Refuge" tavern (NPC interaction)  
Story End: - Confront Riespunzel with the choice to either take the fingernail for the Igel or help him break his own curse, leading to a pivotal moral decision (NPC interaction)


In [18]:
def single_agent_create_scene():
    response = structure_agent.step(
        BaseMessage.make_user_message(
            "User",
            "You need to find 4 to 8 encounters inbetween the first and the last scene."
            "Each step should drive the story forward and offers clues to the next step"
            "They should be ordered according to how the story unfolds"
            "Do not include the first and the last step which are"
            + firstLastResponse.msg.content +
            "For example: "
            "2. Follow the thief through the busteling city alleys (navigation)"
            "3. Sorcerous orbs speaking in riddles in the library (puzzle)"
            "4. Secretive librarian (NPC interaction)"
        )
    )
    print(response.msg.content)
    return response.msg.content


response_content = single_agent_create_scene()

transition_steps = response_content.split("\n")
all_steps = [rows[0], *transition_steps, rows[1]]
print(all_steps)

2. Navigate through the treacherous forest, avoiding giant ants while searching for clues about Riespunzel's whereabouts (navigation)  
3. Encounter a wise old owl who offers cryptic advice about the giant and hints at a hidden path to his cave (NPC interaction)  
4. Solve a puzzle involving ancient forest symbols to unlock a secret passage that leads closer to the giant's cave (puzzle)  
5. Face off against a group of giant ants that guard the entrance to the cave, testing the players' combat skills (combat)  
6. Discover a hidden alcove in the cave containing artifacts that reveal Riespunzel's tragic backstory and the nature of his curse (exploration)  
7. Meet the Eagle, who provides additional information about the Wolf's true intentions and how they might affect the quest (NPC interaction)  
8. Encounter a magical barrier at the cave's entrance that requires teamwork and clever thinking to bypass (puzzle)  
['Story Start: - Gather information about the Igel\'s curse and the giant 

In [19]:
#load example scenes
examples = []
with open("../res/scene-example-1.txt", "r") as file:
    examples.append(file.read())

with open("../res/scene-example-2.txt", "r") as file:
    examples.append(file.read())

with open("../res/scene-example-3.txt", "r") as file:
    examples.append(file.read())
    

In [20]:
# setup scene writer
scene_msg = BaseMessage.make_assistant_message(
    "Scene writer",
    "You write descriptions for scenes played in dnd. "
    "You are writing for the Dungeon Master(DM), not the players. "
    "Remember to make the information easy and fast for the DM to read."
    "The scene you are writing is reference material, not a novel. The DM will be using this live at the table to run games. Split your information out into bullet points and lists, use bolding, get to the point as fast as you can, and cut anything that doesn't matter to the here-and-now (do not explain the history of each room)."
    "Here are examples of how to do it:"
    ""
    "## Example 1:"
    + examples[0] +
    "## Example 2:"
    + examples[1] +
    "## Example 3:"
    + examples[2] +
    "Don't forget you are writing descriptions for dnd scenes"
)
scene_agent = ChatAgent(scene_msg)

In [23]:

dm_msg = BaseMessage.make_assistant_message(
    "Dungeon Master",
    "You are the dungeon master runnng the discussed game. You want to create an emerging and interesting experience for the players."
    "You need to know how the story will unfold. Which steps are optional and which one can be run in different order."
)
dm_agent = ChatAgent(dm_msg)


bm_msg = BaseMessage.make_assistant_message(
    "Beast Master",
    "You know everything about how to balance a good fight and which monsters are allowed in an dnd game."
    "Make sure the right terms are used when adressing monsters or enemies."
)
bm_agent = ChatAgent(bm_msg)

In [26]:
from camel.tasks import Task
from camel.workforce import Workforce


def single_agent_write_scene(prompt):
    scene_response = scene_agent.step(BaseMessage.make_user_message(
        "User",
        prompt
    ))
    return scene_response.msg.content

def workforce_write_scene(prompt):
    workforce = Workforce("Dungeons and Dragons creative team"
                              ## define new_agent_kwargs
                              )
    workforce.add_single_agent_worker(
      "Agent that writes detailed descriptions about scenes",
      worker=scene_agent,
    )
    workforce.add_single_agent_worker(
      "Agent that will use the scene description to run a game of dnd. Acts as a validator for the scenes.",
      worker=dm_agent,
    )
    workforce.add_single_agent_worker(
      "Agent that knows everything about monsters and how to balance fights.",
      worker=bm_agent,
    )
    task = Task(content=prompt)
    response = workforce.process_task(task)
    return response.msg.content
# write a scene
def write_scene(current_scene, next_scene, previous_scene) -> str:
    previous_prompt = "The previous scene contains information and cues that leads to the current scene. Incoorporate these as needed and resolve them. Here is the previous scene: {{" + (previous_scene or "") + "}}. "
    next_prompt = "The scene description should contain information that leads to the next scene. Here is the next scene: {{" + (next_scene or "") + "}}. " if next_scene is not None else ""
    prompt = ("Create a scene description. "
              "Here is the prompt for the current scene you should write a description about: {{" + current_scene + "}}"
              + ( previous_prompt if previous_scene is not None else "")
              + next_prompt  +
              "Do not write a description for the next scene or the previous scene."
              "Don't forget you are writing a description for the current scene."
    )
    scene_response_content = workforce_write_scene(prompt)
    # scene_response_content = single_agent_write_scene(prompt)
    
    print(scene_response_content + "/n")
    
    return scene_response_content

cnt = 0
previous_scene = None
next_scene = None
scene_list = []
for cnt in range(0, len(all_steps) - 1):
    current_scene = all_steps[cnt]
    if cnt != len(all_steps) - 1: 
        next_scene = all_steps[cnt + 1]
        
    current_scene_desc = write_scene(current_scene, next_scene, previous_scene)
    previous_scene = current_scene_desc
    scene_list.append(current_scene_desc)
    cnt += 1
    


RuntimeError: asyncio.run() cannot be called from a running event loop